# LSTM 모델 사용 NLP Classification

1.1 Fully Connected Layer

In [3]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable

class ANN(nn.Module):
    def __init__(self, num_output, input_size, hidden_size, device):
        super(ANN, self).__init__()
        self.device = device
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.outlayer = nn.Linear(hidden_size, num_output)
        
    def forward(self, x):
        h = self.fc1(x).relu()
        h = self.fc2(h).relu()
        predict = sef.outlayer(h)
        return predict

1.2 LSTM for NLP

In [4]:
class LSTM_net(nn.Module):
    def __init__(self, num_output, size_vocab, dim_embed, hidden_size, linear_size, num_layers, device):
        super(LSTM_net, self).__init__()
        self.device = device #GPU
        self.num_output = num_output #1
        self.hidden_size = hidden_size #128
        self.num_layers = num_layers #2
        
        self.embed = nn.Embedding(size_vocab, dim_embed)
        
        self.lstm = nn.LSTM(input_size = dim_embed, hidden_size = hidden_size,
                           num_layers = num_layers, droupout = 0.3, bidirectional = True)
        self.fclayer = nn.Linear(hidden_size, linear_size)
        self.outlayer = nn.Linear(linear_size, num_output)
        
    def forward(self, x):
        scaler = 2 if self.lstm.bidirectional == True else 1
        
        emb = self.embed(x)
        
        h_state = Variable(torch.zeros(self.num_layers*scaler, emb.size(0),
                                      self.hidden_size, requires_grad = True)).to(self.device)
        c_state = Variable(torch.zeros(self.num_layers*scaler, emb.size(0),
                                      self.hidden_size, requires_grad = True)).to(self.device)
        
        lstm_out, (h, c) = self.lstm(emb.transpose(1,0), (h_state, c_state))
        h = h[-1] #important
        h = self.fclayer(h).relu()
        predict = self.outlayer(h)
        return predict